[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/victorkendi/Projeto-2-Vis-Comp/main?labpath=Trabalho2Visualiza%C3%A7%C3%A3oComputacional.ipynb)

# SCC0252 - Visualização Computacional (2022)

### Tema: Preço de combustíveis no Brasil: https://www.kaggle.com/datasets/matheusfreitag/gas-prices-in-brazil

# Equipe:

1.   Deandreson Alves de Souza - 10145310
2.   Gabriel Bueno  11371263
3.   Victor Kendi Arakaki - 11219092



```
# Trabalho de Visualização Computacional
```



In [28]:
#!pip install plotly-express -q
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets, GridspecLayout, Layout

try:
    import google.colab
    from google.colab import output
    IN_COLAB = True
    output.enable_custom_widget_manager()
except:
      IN_COLAB = False

if not IN_COLAB:
    plotly.offline.init_notebook_mode(connected=True)

In [29]:
df = pd.read_table("2004-2021.tsv")

colunas_type = {
    'DATA INICIAL': np.datetime64,
    'DATA FINAL': np.datetime64,
    'REGIÃO': 'category', 
    'ESTADO': 'category', 
    'PRODUTO': 'category',
    'NÚMERO DE POSTOS PESQUISADOS': np.int32, 
    'UNIDADE DE MEDIDA': 'category', 
    'PREÇO MÉDIO REVENDA': np.float64,
    'DESVIO PADRÃO REVENDA': np.float64, 
    'PREÇO MÍNIMO REVENDA': np.float64, 
    'PREÇO MÁXIMO REVENDA': np.float64,
    'MARGEM MÉDIA REVENDA': np.float64, 
    'COEF DE VARIAÇÃO REVENDA': np.float64, 
    'PREÇO MÉDIO DISTRIBUIÇÃO': np.float64,
    'DESVIO PADRÃO DISTRIBUIÇÃO': np.float64, 
    'PREÇO MÍNIMO DISTRIBUIÇÃO': np.float64, 
    'PREÇO MÁXIMO DISTRIBUIÇÃO': np.float64,
    'COEF DE VARIAÇÃO DISTRIBUIÇÃO': np.float64
}
colunas = list(colunas_type.keys())

df[colunas] = df[colunas].replace('-99999.0',0)
df[colunas] = df[colunas].replace('-',0)
df = df.round(2)

for i, t in colunas_type.items():
    #print(i,t)
    df[i] = df[i].astype(t)

In [30]:
metricas = ["preco_medio_revenda", "preco_min_revenda", "preco_max_revenda"]
produtos = ['ETANOL HIDRATADO', 'GASOLINA COMUM']
regioes = df['REGIÃO'].unique()

In [ ]:
df_pais = pd.DataFrame({'DATA INICIAL':df.loc[df['PRODUTO'] == 'ETANOL HIDRATADO', 'DATA INICIAL'].sort_values().unique()})
for produto in produtos:
    df_produto = df.loc[df['PRODUTO'] == produto].sort_values(by=['DATA INICIAL'])
    df_produto['TOTAL PREÇO REVENDA'] = df_produto['PREÇO MÉDIO REVENDA']*df_produto['NÚMERO DE POSTOS PESQUISADOS']
    group = df_produto.groupby(by=['DATA INICIAL']).agg(
    total_preco_revenda=pd.NamedAgg(column="TOTAL PREÇO REVENDA", aggfunc=np.sum),
    total_postos=pd.NamedAgg(column="NÚMERO DE POSTOS PESQUISADOS", aggfunc=np.sum),
    preco_min_revenda=pd.NamedAgg(column="PREÇO MÍNIMO REVENDA", aggfunc="min"),
    preco_max_revenda=pd.NamedAgg(column="PREÇO MÁXIMO REVENDA", aggfunc="max"))
    df_produto = group.reset_index()
    df_produto['preco_medio_revenda'] = (df_produto['total_preco_revenda']/df_produto['total_postos']).round(2)
    df_produto.drop(columns=['total_preco_revenda', 'total_postos'], inplace=True)
    for metrica in metricas:
        df_pais[str(produto+metrica)] = df_produto[metrica]


df_plot = pd.DataFrame({'DATA INICIAL':df_pais['DATA INICIAL']})
for produto in produtos:
    df_plot[produto] = df_pais.loc[:, produto+"preco_medio_revenda"]

In [33]:
dfs_fig2 = []
for produto in produtos:
    df_produto = df.loc[df['PRODUTO'] == produto].sort_values(by=['DATA INICIAL'])
    df_produto['TOTAL PREÇO REVENDA'] = df_produto['PREÇO MÉDIO REVENDA']*df_produto['NÚMERO DE POSTOS PESQUISADOS']
    group = df_produto.groupby(by=['REGIÃO', 'DATA INICIAL']).agg(
    total_preco_revenda=pd.NamedAgg(column="TOTAL PREÇO REVENDA", aggfunc=np.sum),
    total_postos=pd.NamedAgg(column="NÚMERO DE POSTOS PESQUISADOS", aggfunc=np.sum),
    preco_min_revenda=pd.NamedAgg(column="PREÇO MÍNIMO REVENDA", aggfunc="min"),
    preco_max_revenda=pd.NamedAgg(column="PREÇO MÁXIMO REVENDA", aggfunc="max"))
    df_produto = group.reset_index()
    df_produto['preco_medio_revenda'] = (df_produto['total_preco_revenda']/df_produto['total_postos']).round(2)
    df_produto.drop(columns=['total_preco_revenda', 'total_postos'], inplace=True)
    df_regiao = pd.DataFrame({'DATA INICIAL':df_produto['DATA INICIAL']})
    for metrica in metricas:
        df_regiao['Brasil'] = df_pais[str(produto+metrica)]
        for regiao in regioes:  
            df_regiao[regiao] = df_produto[metrica].loc[df_produto['REGIÃO'] == regiao]
            dfs_fig2.append(df_regiao)

In [34]:
dfs_fig3 = []
df_pais2 = pd.DataFrame({'DATA INICIAL':df.loc[df['PRODUTO'] == 'ETANOL HIDRATADO', 'DATA INICIAL'].sort_values().unique()})
for produto in produtos:
    df_produto = df.loc[(df['PRODUTO'] == produto) & (df['PREÇO MÉDIO DISTRIBUIÇÃO'] > 0.0)].sort_values(by=['DATA INICIAL'])
    df_produto['TOTAL PREÇO REVENDA'] = df_produto['PREÇO MÉDIO REVENDA']*df_produto['NÚMERO DE POSTOS PESQUISADOS']
    df_produto['TOTAL PREÇO DISTRIBUIÇÃO'] = df_produto['PREÇO MÉDIO DISTRIBUIÇÃO']*df_produto['NÚMERO DE POSTOS PESQUISADOS']
    group = df_produto.groupby(by=['DATA INICIAL']).agg(
    total_preco_revenda=pd.NamedAgg(column="TOTAL PREÇO REVENDA", aggfunc=np.sum),
    total_preco_distrib=pd.NamedAgg(column="TOTAL PREÇO DISTRIBUIÇÃO", aggfunc=np.sum),
    total_postos=pd.NamedAgg(column="NÚMERO DE POSTOS PESQUISADOS", aggfunc=np.sum))
    df_produto = group.reset_index()
    df_produto['preco_medio_revenda'] = (df_produto['total_preco_revenda']/df_produto['total_postos']).round(2)
    df_produto['preco_medio_distrib'] = (df_produto['total_preco_distrib']/df_produto['total_postos']).round(2)
    df_produto.drop(columns=['total_preco_revenda', 'total_postos', 'total_preco_distrib'], inplace=True)
    df_pais2[str(produto+'preco_medio_revenda')] = df_produto['preco_medio_revenda']
    df_pais2[produto+'preco_medio_distrib'] = df_produto['preco_medio_distrib']


for produto in produtos:
    df_plot = pd.DataFrame({'DATA INICIAL':df_pais2['DATA INICIAL']})
    df_plot[produto+'preco_medio_revenda'] = df_pais2.loc[:, produto+'preco_medio_revenda']
    df_plot[produto+'preco_medio_distrib'] = df_pais2.loc[:, produto+'preco_medio_distrib']
    dfs_fig3.append(df_plot)

In [55]:
f0, f1, f2, f3 = None, None, None, None
def update_f0(tipo_preco, intervalo_anos):
    if tipo_preco == 0:
        tipo = "MÉDIO"
    elif tipo_preco == 1:
        tipo = "MÁXIMO"
    elif tipo_preco == 2:
        tipo = "MÍNIMO"
    
    fig0 = go.Figure()
    fig0.add_trace(go.Violin(x=df['PRODUTO'].loc[(df['PRODUTO'] == 'ETANOL HIDRATADO') & df['DATA INICIAL'].between(intervalo_anos[0]+"-01-01", intervalo_anos[1]+"-12-31")],
                            y=df['PREÇO ' + tipo + ' REVENDA'][df['PRODUTO'] == 'ETANOL HIDRATADO'],
                            name='ETANOL HIDRATADO'))
    fig0.add_trace(go.Violin(x=df['PRODUTO'].loc[(df['PRODUTO'] == 'GASOLINA COMUM') & df['DATA INICIAL'].between(intervalo_anos[0]+"-01-01", intervalo_anos[1]+"-12-31")],
                            y=df['PREÇO ' + tipo + ' REVENDA'][df['PRODUTO'] == 'GASOLINA COMUM'],
                            name='GASOLINA COMUM'))
    fig0.update_traces(box_visible=True)
    fig0.update_layout(
        title_text="Violinplot com Boxplot de " + "preco " + tipo.lower() + " revenda",
        width=500, height=355, 
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    with f0:
        f0.clear_output()
        display(fig0)

def update_f1(tipo_preco,intervalo_anos):
    if tipo_preco == 0:
        tipo = "medio"
    elif tipo_preco == 1:
        tipo = "max"
    elif tipo_preco == 2:
        tipo = "min"

    df_fig1 = pd.DataFrame({'DATA INICIAL':df_pais['DATA INICIAL']})
    for produto in produtos:
        df_fig1[produto] = df_pais.loc[:, produto+"preco_"+tipo+"_revenda"]
    fig1 = px.line(df_fig1.loc[df_fig1['DATA INICIAL'].between(intervalo_anos[0]+"-01-01", intervalo_anos[1]+"-12-31")], x="DATA INICIAL", y=df_fig1.columns,
            hover_data={"DATA INICIAL": "|%B %d, %Y"},
            title='Gráfico de ' + "preco"+tipo+"revenda" + ' no Brasil')
    fig1.update_xaxes(
        dtick="M12",
        tickformat="%Y")
    fig1.update_layout(
        yaxis_title=metrica,
        legend_title="PRODUTO", width=500, height=355,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    with f1:
        f1.clear_output()
        display(fig1)

def update_f2(tipo_preco, tipo_combustivel, intervalo_anos):
    index = tipo_preco + 3*tipo_combustivel
    
    if tipo_preco == 0:
        tipo = "medio"
    elif tipo_preco == 1:
        tipo = "max"
    elif tipo_preco == 2:
        tipo = "min"

    if tipo_combustivel == 0:
        produto = 'ETANOL HIDRATADO'
    elif tipo_combustivel == 1:
        produto = 'GASOLINA COMUM'

    df_plot = dfs_fig2[index].loc[dfs_fig2[index]['DATA INICIAL'].between(intervalo_anos[0]+"-01-01",
                                                                     intervalo_anos[1]+"-12-31")]
    fig2 = px.line(df_plot, x="DATA INICIAL", y=df_plot.columns,
    hover_data={"DATA INICIAL": "|%B %d, %Y"},
    title="Gráfico do " + "preco_" + tipo + "_revenda" + "<br>DE " + produto + " POR REGIÃO")
    fig2.update_xaxes(
        dtick="M12",
        tickformat="%Y")
    fig2.update_layout(
        yaxis_title="preco_" + tipo + "_revenda",
        legend_title="REGIÃO", width=500, height=355)
    with f2:
        f2.clear_output()
        display(fig2)

def update_f3(tipo_combustivel, intervalo_anos):
    if tipo_combustivel == 0:
        produto = 'ETANOL HIDRATADO'
    elif tipo_combustivel == 1:
        produto = 'GASOLINA COMUM'

    df_plot = dfs_fig3[tipo_combustivel].loc[dfs_fig3[tipo_combustivel]['DATA INICIAL'].between(intervalo_anos[0]+"-01-01",
                                                                     intervalo_anos[1]+"-12-31")]
    fig3 = px.line(df_plot, x="DATA INICIAL", y=dfs_fig3[tipo_combustivel].columns,
            hover_data={"DATA INICIAL": "|%B %d, %Y"},
            title='Gráfico do preço médio de<br>' + produto + ' no Brasil')
    fig3.update_xaxes(
        dtick="M12",
        tickformat="%Y")
    fig3.update_layout(
        yaxis_title=metrica,
        legend_title="Tipo", width=500, height=355,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    labels={produto+'preco_medio_revenda': 'Revenda', produto+'preco_medio_distrib': 'Distribuição'}
    fig3.for_each_trace(lambda t: t.update(name = labels[t.name],
                                      legendgroup = labels[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, labels[t.name])
                                     )
                  )
    with f3:
        f3.clear_output()
        display(fig3)

tipo_preco = widgets.ToggleButtons(
    options={'MÉDIO': 0, 'MÁXIMO': 1, 'MÍNIMO': 2},
    description='Speed:',
    disabled=False,
    button_style=''
)
anos = [(str((ano))) for ano in range(2005, 2022)]
style_anos = {}
intervalo_anos = widgets.SelectionRangeSlider(
    options=anos,
    index=(0, 16),
    description='Anos:',
    disabled=False,
    style=style_anos
)

tipo_combustivel = widgets.ToggleButtons(
    options={'ETANOL HIDRATADO': 0, 'GASOLINA COMUM': 1},
    description='Speed:',
    disabled=False,
    button_style=''
)

f0 = widgets.interactive_output(update_f0, {"tipo_preco": tipo_preco, "intervalo_anos": intervalo_anos})
f1 = widgets.interactive_output(update_f1, {"tipo_preco": tipo_preco, "intervalo_anos": intervalo_anos})
f2 = widgets.interactive_output(update_f2, {"tipo_preco": tipo_preco, "tipo_combustivel": tipo_combustivel,
                                            "intervalo_anos": intervalo_anos})
f3 = widgets.interactive_output(update_f3, {"tipo_combustivel": tipo_combustivel, "intervalo_anos": intervalo_anos})


grid_layout = Layout(
    justify_items="center",
)

# criação da grid
grid = GridspecLayout(9, 8, width="1000px", height="800px", layout=grid_layout)

grid[0, 0:3] = tipo_preco
grid[0, 3:5] = intervalo_anos
grid[0, 5:8] = tipo_combustivel
grid[1:5,0:4] = f0
grid[5:9,0:4] = f1
grid[1:5,4:8] = f2
grid[5:9,4:8] = f3

update_f0(0, ["2005", "2021"])
update_f1(0, ["2005", "2021"])
update_f2(0, 0, ["2005", "2021"])
update_f3(0, ["2005", "2021"])

grid

GridspecLayout(children=(ToggleButtons(description='Speed:', layout=Layout(grid_area='widget001'), options={'M…